In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import xml.etree.ElementTree as ET
import os,sys

- Function to caculate average precision given precision and recall at each point

In [2]:
def compute_average_prec(rec, prec):
    
    # Correct Method
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
         mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

- Parse the annotaion file and returns list of objects (each object is a dict) present in the image 

In [3]:
def parse_rec(filename):
    """ Parse a stateDetection xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text),
                              int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text),
                              int(bbox.find('ymax').text)]
        objects.append(obj_struct)

    return objects

- Read all annotations 
- save it in dictionary (key: imagename, val: list of dictionary representing all objects of image)

In [4]:
# path to image set file name train/val.txt contains image name in each line
imagesetfile = '/home/ashu/Study Material/Uni Bonn/4th Semester/Thesis/DataSet/coco-master/ImageSets/Main/val.txt'

with open(imagesetfile, 'r') as f:
        lines = f.readlines()
imagenames = [x.strip() for x in lines]

annopath = os.path.join(
            'Annotations',
            '{:s}.xml')
#load all annotations 
recs = {}
for i, imagename in enumerate(imagenames):
    recs[imagename] = parse_rec(annopath.format(imagename))
    if i % 1000 == 0:
        print 'Reading annotation for {:d}/{:d}'.format(
                i + 1, len(imagenames))
print len(recs)

Reading annotation for 1/4874
Reading annotation for 1001/4874
Reading annotation for 2001/4874
Reading annotation for 3001/4874
Reading annotation for 4001/4874
4874


- class_recs : contains filtered gt data for the specified class only

In [153]:
#state_name = 'openLaptop'
class_name = 'umbrella'
#Extract gt objects of this state only
class_recs = {}
npos = 0
for imagename in imagenames:
        R = [obj for obj in recs[imagename] if obj['name'].split('_')[0] == class_name]
        
        bbox = np.array([x['bbox'] for x in R])
        det = [False] * len(R)
        
        difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
        npos = npos + sum(~difficult)
        class_recs[imagename] = {'bbox': bbox, # all gt bbox of image
                                 'difficult': difficult,  # every object is marked as not difficult
                                 'det': det}   # mark as detected or not

- Just for sanity check no of bbox in specified class must be equal to number of objects in val set

In [154]:
# check number of bbox in class, must be equal to number of objects in val set of that class
count = 0
for imagename in imagenames:
    R = class_recs[imagename]
    BBGT = R['bbox'].astype(float)
    if BBGT.size > 0: # i.e. image contains the box
        count += len(BBGT)
print 'Total objects in ', class_name, ': ', count

Total objects in  umbrella :  3935


- specify the generated detections (Open / Close) file

In [155]:
# Root folder for Detections
classifier_type = 'gen'
caffe_root = '/home/ashu/Desktop/Thesis Work/Classifier/caffe/'
images_root = '/home/ashu/Study Material/Uni Bonn/4th Semester/Thesis/DataSet/coco-master/JPEGImages/'

if class_name == 'scissors':
    class_name = 'scissor'

det_file = 'cls_det_frm_state/concat_fc7_cls_bbox/' + class_name + '.txt'
#det_file = caffe_root + 'models/gt_classifier/gen_' + class_name + '/detections/'  \
#            + classifier_type + '_val_' + state_name + '.txt'
    
print det_file

cls_det_frm_state/concat_fc7_cls_bbox/umbrella.txt


- Read the detection and remove new line at end of each scentece from it

In [156]:
# load detections
#open_file = os.path.join('det_val','det_val_laptop_closedLaptop.txt')
#close_file = os.path.join('det_val','det_val_laptop_openLaptop.txt')

with open(det_file, 'r') as f:
    lines = np.array(f.readlines())

remove_new_line = np.vectorize(lambda x: x.strip())

lines = remove_new_line(lines)

print lines[:3], lines[-3:]

['COCO_val2014_000000568337 0.268 22.0 70.2 162.8 154.8'
 'COCO_val2014_000000568337 0.154 156.4 227.8 351.7 399.5'
 'COCO_val2014_000000568337 0.089 33.9 65.5 362.7 427.9'] ['COCO_val2014_000000210342 0.141 261.1 117.5 551.5 318.7'
 'COCO_val2014_000000210342 0.063 518.3 357.4 541.4 401.4'
 'COCO_val2014_000000534784 0.051 356.8 303.8 383.2 349.3']


- Loop through detection
- calculate IoU overlap
- Mark tp and fp
- calulate prec and recall at each point

In [157]:
ovthresh = 0.5
splitlines = [x.strip().split(' ') for x in lines]
image_ids = [x[0] for x in splitlines]
confidence = np.array([float(x[1]) for x in splitlines])
BB = np.array([[float(z) for z in x[2:6]] for x in splitlines])

# sort by confidence
sorted_ind = np.argsort(-confidence)
sorted_scores = np.sort(-confidence)
BB = BB[sorted_ind, :]
image_ids = [image_ids[x] for x in sorted_ind]

# go down dets and mark TPs and FPs
nd = len(image_ids)
tp = np.zeros(nd)
fp = np.zeros(nd)

print nd
# looping through detections
for d in range(nd):
    R = class_recs[image_ids[d]]
    bb = BB[d, :].astype(float) # Detected bbox
    ovmax = -np.inf
    BBGT = R['bbox'].astype(float) # GT bboxes for same image

    if BBGT.size > 0:
        # compute overlaps
        # intersection
        ixmin = np.maximum(BBGT[:, 0], bb[0])
        iymin = np.maximum(BBGT[:, 1], bb[1])
        ixmax = np.minimum(BBGT[:, 2], bb[2])
        iymax = np.minimum(BBGT[:, 3], bb[3])
        iw = np.maximum(ixmax - ixmin + 1., 0.)
        ih = np.maximum(iymax - iymin + 1., 0.)
        inters = iw * ih

        # union
        uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
               (BBGT[:, 2] - BBGT[:, 0] + 1.) *
                (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

        overlaps = inters / uni
        ovmax = np.max(overlaps)  # taking the gt bbox that has max overlap
        jmax = np.argmax(overlaps)  # taking the index of max overlap bbox
    
    # check if IoU crosses threshold
    if ovmax > ovthresh:
        if not R['difficult'][jmax]:
            if not R['det'][jmax]:
                tp[d] = 1.
                R['det'][jmax] = 1 # marking that bbox as detected
            else:
                fp[d] = 1
    else:
       fp[d] = 1 

fp = np.cumsum(fp)
tp = np.cumsum(tp)
rec = tp / float(npos)
# avoid divide by zero in case the first detection matches a difficult
prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)

23115


- calculate the avergae precision.

In [158]:
ap = compute_average_prec(rec, prec)
print 'Average Precision for ' + class_name + ' : ', ap

Average Precision for umbrella :  0.35333292225
